<a href="https://colab.research.google.com/github/AustinJamesWolff/Business-Projects/blob/main/Company_CRM_Unscrambling_Records.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **The Problem:**
#### When The Company (kept anonymous for privacy) began using the HighLevel CRM ("HighLevel"), data had to exported from the old CRM, AgileCRM ("Agile"), and imported into HighLevel. **Apparently, a few phone numbers and emails were switched between contacts during the import process for an unknown reason, and seemingly at random.**

It's likely the unwanted "phone number swap" happened due to an unknown error with HighLevel, however, this bug was noticed by the sales team a few months after the import, and the HighLevel tech team only keeps logs of imports for up to one month. So now, they need help with the following:

- Determining which contacts had their phone numbers or emails swapped (**bugContacts**),
- Matching the phone numbers and emails to the correct contact,
- Creating a new spreadsheet with only the corrected contacts (**cleanContacts**)
- Importing the **cleanContacts** back into HighLevel in a way which overwrites only the wrong phone numbers, and does not change any other data already conencted with the contact,
  - For example, importing the contacts with the correct phone numbers should simply overwrite wrong phone numbers, and nothing else such as name, email, notes, etc.

**Side Note:**

- If overwriting one record at a time was possible, the salesmen would have just corrected the data each time they found an error, but this bug was like a full web of contacts with their information all intertwined with each other. So you had to find out how many contacts were intertwined with each other, then you had to change the entire web of contacts at once rather than individually.

<br>



### **Brainstormed Possible Solution:**
- Import both csv files of HighLevel and AgileCRM
- Create a new dataframe containing only names and emails which match both datasets, **and** where the phone numbers do not match,
- Columns should be as followed:
  - highlevel_fullname
  - highlevel_email
  - highlevel_phone
  - agile_fullname
  - agile_email
  - agile_phone
- Records from Agile are correct, so overwrite *highlevel_phone* with *agile_phone*
- Create new DataFrame, **cleanedHighLevel** with the following:
  - highlevel_fullname
  - highlevel_email
  - highlevel_phone
- Export new spreadsheet, **cleanedHighLevel.csv**
- Import **cleanedHighLevel.csv** into HighLevel, and overwrite the phone numbers of all existing contacts in HighLevel with those in the spreadsheet.

<br>


### **How The Problem Was Actually Solved:**

The notebook below details the exact steps taken to fix their CRM. Here is a brief summary:

  - I imported the entire dataset from Highlevel and Agile (their new and old CRMs, respectively) and cleaned the data. Then I organized it in a way to easily comapre the two datasets with each each other.
 
  - Then I merged both datasets into a dataframe with all contacts who shared the same email (a "same-email" list), but otherwise have different names and/or phone numbers on both CRMs. 

  - Then I created a 2nd dataframe with all contacts who shared the same phone number, but had different names and/or emails (a "same-phone" list). 

  - Most of the records in the "same-phone" list were different in Highlevel on purpose: the Highlevel CRM is constantly updated, while AgileCRM is no longer updated. This means that the majority of the data inconsistency turned out to simply be the salesmen updating contact info as they went along, and these records could be ommitted from our dataset-cleaning process.

  - On the "same-phone" list (about 160 contacts), I noticed the bottom 24 seemed to match the 24 contacts in the "same-email," just organized differently. Not only that, I noticed a pattern of data scrambling on the bottom 24 records of both lists (the pattern is too complicated to summarize here). 

  - There were a few outlier records which did not fit the pattern, making it seem much more complex than it really was. Once I manually scrubbed the outliers off the list, the pattern became much more straightforward.

  - Thanks to this pattern, I was able to detect where holes of data where, which eventually led me to the exact records in the new CRM which were scrambled.  
  
  - I re-exported the correct data from the old CRM and simply overwrote the new CRM data on those 24 records. I then checked each one individually to double-check the work (this double-checking took less than 5 minutes). 

<br>


### **Conclusion:**
  
  - The CRM is now fully functional. Before I worked on this, it was unknown how many records were swapped, and it couldn't be done individually. A full end-to-end solution had to be thought of and all data had to be overwriten at once to overcome the constraints of the software. Now the salesmen can happily focus on closing more leads rather than worrying if their leads' contact information is scrambled.

<br>


### **Tools Used:**

  - Python, Google Colab, Pandas Library, DataPrep Library, Google Sheets, HighLevel CRM, AgileCRM




In [ ]:
# An extra line will be added to each code block for readability
from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:
from google.colab import files
files.upload()


{}

In [74]:
import pandas as pd

# Import both csv files of HighLevel and AgileCRM
od_highlevel = pd.read_csv('/content/drive/MyDrive/Data Science Projects/Company_CRM_Dataset_Comparison/oldHighlevel.csv', dtype = str)
old_agile = pd.read_csv('/content/drive/MyDrive/Data Science Projects/Company_CRM_Dataset_Comparison/oldAgile.csv', dtype = str)


In [ ]:
# Drop columns in old_highlevel which aren't name, phone, or email
old_highlevel.drop(["Index","Business Name","Created","Last Activity","Tags","Unnamed: 9"], axis=1, inplace=True)


In [ ]:
# Test dropped columns
old_highlevel.head()


,First Name,Last Name,Phone,Email
0,Ruben,Avalos,15097781007,rubenavalos1981@gmail.com
1,Frank,Cotton,13233823603,fcottonjr@gmail.com
2,Erna,Bright,12523399048,ernabrightjr@embarqmail.com
3,David,Hunt,1617448792128,djhunt@bigpond.net.au
4,Ghl,NaN,NaN,dipesh@istesting.app


In [ ]:
# Rename old_agile columns to match old_highlevel
old_agile.rename(columns={"Email(default)":"Email","Phone(default)":"Phone"}, inplace=True)


In [ ]:
# Combine all phone columns in old_agile (the CRM had multiple places to input phone numbers)
old_agile['Phone'].fillna(old_agile['Phone(Work)'], inplace=True)
old_agile['Phone'].fillna(old_agile['Phone(Mobile)'], inplace=True)


In [ ]:
# Test dropped columns
old_agile = old_agile[["First Name","Last Name","Phone","Email"]]
old_agile.head()


,First Name,Last Name,Phone,Email
0,David,Cassis,+1818-391-4975,dollarkingdave@gmail.com
1,Henry,Tof,83146397116,ivanovnaang@gmail.com
2,Travis,Forsey,+1201-379-4216,tdors8719@gmail.com
3,Michael,Reppucci,310-500-0000,NaN
4,Bruce,Wilson,+19496904760,brucescvn@gmail.com


In [ ]:
# Columns in old_highlevel should be renamed as followed:
# highlevel_firstname
# highlevel_lastname
# highlevel_email
# highlevel_phone
old_highlevel = old_highlevel.rename(
    columns={
        "First Name":"highlevel_firstname",
        "Last Name":"highlevel_lastname",
        "Phone":"highlevel_phone",
        "Email":"highlevel_email"
    }
)


In [ ]:
# Test
old_highlevel.head()


,highlevel_firstname,highlevel_lastname,highlevel_phone,highlevel_email
0,Ruben,Avalos,15097781007,rubenavalos1981@gmail.com
1,Frank,Cotton,13233823603,fcottonjr@gmail.com
2,Erna,Bright,12523399048,ernabrightjr@embarqmail.com
3,David,Hunt,1617448792128,djhunt@bigpond.net.au
4,Ghl,NaN,NaN,dipesh@istesting.app


In [ ]:
# Columns in old_agile should be renamed as followed:
# agile_firstname
# agile_lastname
# agile_email
# agile_phone
old_agile = old_agile.rename(
    columns={
        "First Name":"agile_firstname",
        "Last Name":"agile_lastname",
        "Phone":"agile_phone",
        "Email":"agile_email"
    }
)


In [ ]:
# Test
old_agile.head()

,agile_firstname,agile_lastname,agile_phone,agile_email
0,David,Cassis,+1818-391-4975,dollarkingdave@gmail.com
1,Henry,Tof,83146397116,ivanovnaang@gmail.com
2,Travis,Forsey,+1201-379-4216,tdors8719@gmail.com
3,Michael,Reppucci,310-500-0000,NaN
4,Bruce,Wilson,+19496904760,brucescvn@gmail.com


In [ ]:
# This is necessary to use the DataPrep library
pip install -U dataprep


In [ ]:
# Import DataPrep library to clean phone numbers
from dataprep.clean import clean_phone


In [ ]:
# Format the phone numbers the same
old_agile = clean_phone(old_agile, 'agile_phone', output_format='national', inplace=True)


Phone Number Cleaning Report:
	8518 values cleaned (86.3%)
	520 values unable to be parsed (5.27%), set to NaN
Result contains 8618 (87.32%) values in the correct format and 1252 null values (12.68%)


In [ ]:
# Format the phone numbers the same
old_highlevel = clean_phone(old_highlevel, 'highlevel_phone', output_format='national', inplace=True)


Phone Number Cleaning Report:
	8510 values cleaned (88.83%)
	27 values unable to be parsed (0.28%), set to NaN
Result contains 8510 (88.83%) values in the correct format and 1070 null values (11.17%)


In [ ]:
# Remove all relevant (email and phone) NaN values
old_highlevel = old_highlevel[old_highlevel['highlevel_phone_clean'].notna()]
old_highlevel = old_highlevel[old_highlevel['highlevel_email'].notna()]
old_agile = old_agile[old_agile['agile_phone_clean'].notna()]
old_agile = old_agile[old_agile['agile_email'].notna()]


In [ ]:
# Check for relevant NaN values
print(old_highlevel.isnull().sum())
print(old_agile.isnull().sum())

highlevel_firstname        0
highlevel_lastname       537
highlevel_email            0
highlevel_phone_clean      0
dtype: int64
agile_firstname        1
agile_lastname       431
agile_email            0
agile_phone_clean      0
dtype: int64


In [ ]:
# Easy viewing of both lists if you uncomment
# old_highlevel.head()
# old_agile.head()
# old_agile.shape
old_highlevel.shape


(8135, 4)

In [ ]:
# Check to see if email records in old_agile are in old_highlevel
old_agile_matched_emails = old_agile[old_agile['agile_email'].isin(old_highlevel['highlevel_email'])]


In [ ]:
# Test
# old_agile_matched_emails.head()
old_agile_matched_emails.shape


(6241, 4)

In [ ]:
# New dataframe for only emails in HighLevel that are also in Agile
old_highlevel_matched_emails = old_highlevel[old_highlevel['highlevel_email'].isin(old_agile_matched_emails['agile_email'])]


In [ ]:
# Test
old_highlevel_matched_emails.shape

(6286, 4)

In [ ]:
# Rename so they all have matching column names
old_agile_matched_emails = old_agile_matched_emails.rename(
    columns = {
        "agile_email":"EMAIL"
    }
)


In [ ]:
old_highlevel_matched_emails = old_highlevel_matched_emails.rename(
    columns = {
        "highlevel_email":"EMAIL"
    }
)


In [ ]:
# Test
old_agile_matched_emails.head()


,agile_firstname,agile_lastname,EMAIL,agile_phone_clean
0,David,Cassis,dollarkingdave@gmail.com,(818) 391-4975
2,Travis,Forsey,tdors8719@gmail.com,(201) 379-4216
4,Bruce,Wilson,brucescvn@gmail.com,(949) 690-4760
5,Brandon,Cruz,brandoncruz769@yhaoo.com,(678) 824-3211
6,Gustavo,Reygadas,gus.reygadas@gmail.com,(626) 419-5070


In [ ]:
# Test inner_merge with a test dataframe variable
test_EMAIL_dataframe = pd.merge(old_highlevel_matched_emails, old_agile_matched_emails, how="outer", on=['EMAIL'])


In [ ]:
# Drop duplicates
test_EMAIL_dataframe = test_EMAIL_dataframe.drop_duplicates(keep="first")


In [ ]:
# Test views of the new DataFrame. Just uncomment.
# test_EMAIL_dataframe.shape
# test_EMAIL_dataframe.tail(50)
# test_EMAIL_dataframe[6190:6250]


In [ ]:
# Create a new Dataframe where only keep rows if ['highlevel_phone_clean'] != ['agile_phone_clean']
different_phone_dataframe = test_EMAIL_dataframe[test_EMAIL_dataframe['highlevel_phone_clean'] != test_EMAIL_dataframe['agile_phone_clean']]


In [ ]:
# This is to drop any "outlier records" we have ether already fixed or ruled out,
# because time has passed and the Highlevel dataset is much more up to date

# different_phone_dataframe = different_phone_dataframe[different_phone_dataframe['EMAIL'] != 'arenamike1950@gmail.com']
# different_phone_dataframe = different_phone_dataframe[different_phone_dataframe['EMAIL'] != 'e.t.tiles@hotmail.com']
# different_phone_dataframe = different_phone_dataframe[different_phone_dataframe['EMAIL'] != 'lmason1952@yahoo.com']
# different_phone_dataframe = different_phone_dataframe[different_phone_dataframe['EMAIL'] != 'oscarmendoza1955@gmail.com']
# different_phone_dataframe = different_phone_dataframe[different_phone_dataframe['EMAIL'] != 'austinwolff88@gmail.com']
different_phone_dataframe = different_phone_dataframe[different_phone_dataframe['highlevel_lastname'].notna()]


In [ ]:
# Test 
different_phone_dataframe
# different_phone_dataframe.shape


,highlevel_firstname,highlevel_lastname,EMAIL,highlevel_phone_clean,agile_firstname,agile_lastname,agile_phone_clean
268,Micheal,Arena,arenamike1950@gmail.com,(661) 486-3152,Micheal,Arena,(661) 486-4152
2148,Robert,Enciso,e.t.tiles@hotmail.com,(818) 472-5355,Robert,Enciso,(818) 365-2389
2832,Lee,Mason,lmason1952@yahoo.com,(323) 381-5799,Lee,Mason,(323) 998-2043
5978,Oscar,Millan,oscarmendoza1955@gmail.com,(818) 213-0287,Oscar,Millan,(818) 800-6032
6154,Testing,Wolff,austinwolff88@gmail.com,(818) 400-9130,TEST,Test,(818) 400-1930
6209,Angel,Basdeo,farwestair@gmail.com,(608) 807-9292,Jon,Hunsaker,(503) 507-4003
6210,Rebecca,Hunsaker,mexicoar2@hotmail.com,(323) 300-6023,Angel,Reynoso,(313) 812-5969
6212,Roman,Reynoso,rebecstar1@icloud.com,(323) 868-4928,Rebecca,Rininger,(240) 600-7644
6214,Jorge,Rininger,coloradolife97@gmail.com,(626) 945-7555,Roman,Mayers,(608) 807-9292
6216,Patrick,Mayers,jrramosusa@yahoo.com,(213) 840-9007,Jorge,Ramos,(323) 300-6023


We have just created our "email-list," meaning a merging of the to CRMs based on contact email, but only for contacts whose phone numbers do not match each other. 

We will now do the same below, but this time, creating a "phone-list," meaning a merging of lists based on contact phone numbers, but only for contacts whose emails do not match the original.

In [ ]:
# Create a new Dataframe for all rows that match phone numbers
old_agile_matched_phone = old_agile[old_agile['agile_phone_clean'].isin(old_highlevel['highlevel_phone_clean'])]
old_highlevel_matched_phone = old_highlevel[old_highlevel['highlevel_phone_clean'].isin(old_agile_matched_phone['agile_phone_clean'])]


In [ ]:
# Test
old_agile_matched_phone
# old_highlevel_matched_phone

,agile_firstname,agile_lastname,agile_email,agile_phone_clean
0,David,Cassis,dollarkingdave@gmail.com,(818) 391-4975
2,Travis,Forsey,tdors8719@gmail.com,(201) 379-4216
4,Bruce,Wilson,brucescvn@gmail.com,(949) 690-4760
5,Brandon,Cruz,brandoncruz769@yhaoo.com,(678) 824-3211
6,Gustavo,Reygadas,gus.reygadas@gmail.com,(626) 419-5070
...,...,...,...,...
9863,Joe,Riehm,riehmjoe@yahoo.com,(818) 357-0533
9864,Austin,Wolff,austin+080620-1@thenovuscenter.com,(818) 400-9130
9867,Jorge,Ramos,jrramosusa@yahoo.com,(323) 300-6023
9868,Greg,DeLong,gkdelo@gmail.com,(757) 232-7731


In [ ]:
# Rename so they all have matching column names
old_agile_matched_phone = old_agile_matched_phone.rename(
    columns = {
        "agile_phone_clean":"PHONE"
    }
)

old_highlevel_matched_phone = old_highlevel_matched_phone.rename(
   columns = {
       "highlevel_phone_clean":"PHONE"
   } 
)


In [ ]:
# Merge them togehter in new dataframe
test_phone_dataframe = pd.merge(old_highlevel_matched_phone, old_agile_matched_phone, how="outer", on=['PHONE'])


In [ ]:
# Drop duplicates
test_phone_dataframe = test_phone_dataframe.drop_duplicates(keep="first")


In [ ]:
# test
test_phone_dataframe

,highlevel_firstname,highlevel_lastname,highlevel_email,PHONE,agile_firstname,agile_lastname,agile_email
0,Ruben,Avalos,rubenavalos1981@gmail.com,(509) 778-1007,Ruben,Avalos,rubenavalos1981@gmail.com
1,Paris,Londi,prcoach248@gmail.com,(626) 389-7914,Paris,Londi,prcoach248@gmail.com
2,Albert,Melgar,albertomelgar12@yayoo.com,(323) 217-9740,Albert,Melgar,albertomelgar12@yayoo.com
3,David,S.,wlanative@aol.com,(818) 478-6068,David,S.,wlanative@aol.com
4,Terry,Ally,terryally73@gmail.com,(310) 892-6120,Terry,Ally,terryally73@gmail.com
...,...,...,...,...,...,...,...
6417,Robert,Leduc,rleduc34@gmail.com,(323) 447-2844,Robert,Leduc,rleduc34@gmail.com
6420,Marc,Allen,frenchmen90631@gmail.com,(562) 414-9207,Marc,Allen,frenchmen90631@gmail.com
6422,Sam,Dabach,55dabach@gmail.com,(818) 943-7771,Sam,Dabach,55dabach@gmail.com
6424,Gabriel,Alvarado,aaalvaradoh@hotmail.com,(626) 873-5652,Victor Gabriel,Alvarado,aaalvaradoh@hotmail.com


In [ ]:
# New dataframe if the rows have different emails but the same phone number
different_email_dataframe = test_phone_dataframe[test_phone_dataframe['highlevel_email'] != test_phone_dataframe['agile_email']]


In [ ]:
#Test
different_email_dataframe


,highlevel_firstname,highlevel_lastname,highlevel_email,PHONE,agile_firstname,agile_lastname,agile_email
26,Joe,Cruz,joebcruz@gmail.com,(626) 701-6644,Joe,Cruz,joebcruz@gmsil.com
179,Nazmul,Kahn,kahn.nash@yahoo.com,(818) 281-7193,Nazmul,Kahn,nazmul.kahn@gmail.com
220,Jim,Torp,jumtorp07@gmail.com,(661) 510-4183,Jim,Torp,jimtorp07@gmail.com
242,May,Lin,mayl0105@yahoo.com,(626) 731-8398,May,Lin,mayl105@yahoo.com
243,Juliana,E,jencinas29@gmail.com,(555) 555-5555,Test,NaN,testing@test.com
...,...,...,...,...,...,...,...
6379,Jesus,Wolff,moraleschato23@gmail.com,(424) 413-9449,Steve,Newberry,lexi4now68@gmail.com
6381,Ed,Marquez,ecg1109@gmail.com,(949) 292-3343,Kevin,Mowry,kevinmowry@cox.net
6383,Steve,Gonzales,lexi4now68@gmail.com,(908) 670-6120,Drew,Brendli,drewbrendli@thenovuscenter.com
6404,Ulrich,Huegli,swisstravelchef@yahoo.com,(818) 970-4896,Ueli,Huegli,matterchef1950@yahoo.com


In [ ]:
# Export different_email_dataframe
different_email_dataframe.to_csv('same_phone_different_email.csv')


In [ ]:
# Export different_phone_dataframe
# Any spreadsheet application offers a quicker way for me to visual and parse data that isn't too big (and can be analyzed within a few minutes)
# Luckily, after reviewing "different_email_dataframe," there was only a count of 24 records that were scrambled with the discovered pattern.
# The other 100+ records in that dataframe didn't match on purpose, as the Highlevel data is constantly updated while Agile's is not
# Therefore, I will export different_phone_dataframe and see if the records here match the exact pattern and indexing as different_email_dataframe

different_phone_dataframe.to_csv('same_email_different_phone.csv')


At this point, I uploaded the csv files into Google Sheets for me to quickly visualize and move around cells or rows based on the pattern that began to emerge after scrubbing outliers. Once the pattern was fully recognized, I was able to quickly group together the "web of contacts" in both CRMs. I tagged them 'reupload' in both programs, exported the original data from AgileCRM, and reuploaded it to overwrite the data in Highlevel CRM. I double-checked each contact to ensure the problem was solved. Indeed it was.

<br>

**Austin Wolff**

**9/2/21**